In [4]:
import pathlib
import json

import sys
sys.path.append("/scr/ig_pipeline")

from b1k_pipeline.utils import parse_name

# Go through each target
for target in pathlib.Path("/scr/ig_pipeline").rglob("cad/*/*"):
    # Load the backup file
    bkp_path = target / "artifacts/collision_selection_backup.json"
    backup_selection = {}
    if bkp_path.exists():
        backup_selection = json.loads(bkp_path.read_text())

    # Previous selection ID mapping
    key_to_entry = {}
    for entry in backup_selection.keys():
        pn = parse_name(entry)
        key = (pn.group("model_id"), pn.group("link_name"))
        key_to_entry[key] = entry

    # Load the current file
    cur_path = target / "artifacts/collision_selection.json"
    current_selection = {}
    if cur_path.exists():
        current_selection = json.loads(cur_path.read_text())

    # Combine the two
    combined_selection = {}
    combined_selection.update(backup_selection)
    
    for entry, value in current_selection.items():
        pn = parse_name(entry)
        key = (pn.group("model_id"), pn.group("link_name"))
        if key in key_to_entry:
            combined_selection[key_to_entry[key]] = value
        else:
            combined_selection[entry] = value

    # Remove the backup file
    if bkp_path.exists():
        bkp_path.unlink()

    # Write the combined file
    with open(cur_path, "w") as f:
        json.dump(combined_selection, f, indent=2)

FileNotFoundError: [Errno 2] No such file or directory: '/scr/ig_pipeline/cad/objects/jersey-of/artifacts/collision_selection.json'